# **Spatial typology to identify food and nutrition security bottlenecks in Niger**

**IFPRI- Calculate Productive Agriculture :Arable land computation**


```bash
Author : Aboubacar HEMA
Supervision : Wim MARIVOET
Contact : a.hema@cgiar.org / w.marivoet@cgiar.org
Role : Research Analyst at IFPRI
Year : December 2022
```

-----------------
## **Objective**
-----------------

- To identify arable land in country of interest at 30m resolution, comprising the following steps: 
1. Aggregate Cleared Forest and Cropland datasets 
2. Masking slope > 15%
3. permanent water
4. imprevious surfaces 


-----------------
## **Output**
-----------------



-----------------
## **Dataset**
-----------------


 Directions for Downloading GFSAD Data:
1. Go to https://search.earthdata.nasa.gov/search. If you're a new user, create a login using contact information.
2. Search "GFSAD" in the top left search bar "Search for collections and topics"
3. Select "Global Food Security-support Analysis Data (GFSAD) Cropland Extent 2015 Africa 30 m V001"
4. In the map area to the right, image footprints will appear as on overlay over Africa. Navigate to country of interest, select image footprint that you would like to download. In the left side table showing search results, the image you have selected will be outlined in green (NOTE: This may require scrolling through the image list).
5. In the green outlined box, click "Download Granule" to download the scene to the desired output location on your computer.
6. Repeat these steps until you have downloaded all images required to cover your entire area of interest.
7. Upload download GFSAD geoTIFFs to GEE from Assets and import them into calcArableLand script.![image](https://userimages.githubusercontent.com/54441886/206823358-5993aad6-ae09-40d6-a33f-b14dd4f04bc7.png)

-----------------
## **Technical requirements**
-----------------


```bash
conda create -n gee python
conda activate gee
conda install -c conda-forge mamba
mamba install -c conda-forge pygis
pip install geemap
pip install ee
```


-----------------
## **Import the necessary libraries**
-----------------


In [1]:
import ee
import geemap

# Basic libraries of python for numeric and dataframe computations
import numpy as np                              
import pandas as pd

# Basic library for data visualization
import matplotlib.pyplot as plt 

# Slightly advanced library for data visualization            
import seaborn as sns 

In [2]:
# Authenticate and Initialize Earth Engine
geemap.ee_initialize()

-----------------
## **Load data**
-----------------

In [4]:
Hansen_GFC = ee.Image("UMD/hansen/global_forest_change_2020_v1_8")
SRTM = ee.Image("USGS/SRTMGL1_003")
surfaceWater = ee.Image("JRC/GSW1_3/GlobalSurfaceWater")
imperviousSurface = ee.Image("Tsinghua/FROM-GLC/GAIA/v10")
geometry =ee.Geometry.Polygon(
        [[[-2.079997168246366, 24.514922224888306],
          [-2.079997168246366, 11.423798195385235],
          [19.101643456753635, 11.423798195385235],
          [19.101643456753635, 24.514922224888306]]])
GFSAD30AFCE_2015_N_1 = ee.Image("projects/ee-aboubacarhema94/assets/Niger/GFSAD30AFCE_2015_N_1")
GFSAD30AFCE_2015_N_2 = ee.Image("projects/ee-aboubacarhema94/assets/Niger/GFSAD30AFCE_2015_N_2")
GFSAD30AFCE_2015_N_3 = ee.Image("projects/ee-aboubacarhema94/assets/Niger/GFSAD30AFCE_2015_N_3")
GFSAD30AFCE_2015_N_4 = ee.Image("projects/ee-aboubacarhema94/assets/Niger/GFSAD30AFCE_2015_N_4")
ImperviousSurfaceGMIS = ee.Image("projects/ee-aboubacarhema94/assets/Niger/NER_gmis_impervious_surface_percentage_geographic_30m")
ProtectedAreaPoints = ee.FeatureCollection("projects/ee-aboubacarhema94/assets/Niger/ProtectedAreaPoints")
ProtectedAreaPolygons = ee.FeatureCollection("projects/ee-aboubacarhema94/assets/Niger/ProtectedAreaPolygons")
GAUL_adm0 = ee.FeatureCollection("projects/ee-aboubacarhema94/assets/Niger/gadm41_NER_0")
GAUL_adm1 = ee.FeatureCollection("projects/ee-aboubacarhema94/assets/Niger/gadm41_NER_1")
GAUL_adm2 = ee.FeatureCollection("projects/ee-aboubacarhema94/assets/Niger/gadm41_NER_2")
GAUL_adm0F = ee.FeatureCollection("FAO/GAUL/2015/level0")
GAUL_adm2F = ee.FeatureCollection("FAO/GAUL/2015/level2")
GAUL_adm3 = ee.FeatureCollection("projects/ee-aboubacarhema94/assets/Niger/gadm41_NER_3")
NER_adm2 = ee.FeatureCollection("projects/ee-aboubacarhema94/assets/Niger/NER_64")

In [26]:
deforestCompYear = 14 #Last two digits of year only
occurrence_threshold = 10 #Set water occuring less than this value (in %) of the time, it is the frequency with which water was present.


In [8]:
adm0_AOI = GAUL_adm0
#Initialize a Map
Map = geemap.Map()
Map.centerObject(adm0_AOI)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [6]:
# Set visualization/style parameters
adm2_style = {
    "fillColor": "00000000", # transparent color code
    "color": "black", # color of the stroke
    "width": 0.5 # stroke width
}

# Display the layer
Map.addLayer(NER_adm2.style(**adm2_style), {}, " Niger Administrative two Boundaries")
Map

Map(bottom=7677.0, center=[17.6150789448675, 8.08094346368385], controls=(WidgetControl(options=['position', '…


-----------------
## **Global Food Security-support Analysis Data (GFSAD) Cropland Extent 2015 Africa 30 m V001**
-----------------

In [14]:
GFSAD_1 = GFSAD30AFCE_2015_N_1
GFSAD_2 = GFSAD30AFCE_2015_N_2
GFSAD_3 = GFSAD30AFCE_2015_N_3
GFSAD_4 = GFSAD30AFCE_2015_N_4
#CLip Images to Niger Extent --> Mask
clipGFSAD_1 = GFSAD_1.clip(adm0_AOI)
clipGFSAD_2 = GFSAD_2.clip(adm0_AOI)
clipGFSAD_3 = GFSAD_3.clip(adm0_AOI)
clipGFSAD_4 = GFSAD_4.clip(adm0_AOI)
#Create Image Collection so it can be mosaiced together --> use quality mosaic to get highest
GFSAD = ee.ImageCollection([clipGFSAD_1, clipGFSAD_2, clipGFSAD_3, clipGFSAD_4]).mosaic()
#Mask to select Cropland (band b1 = 2)
aoiCropland = GFSAD.eq(2).selfMask().rename("Cropland")

In [12]:
bin_viz = {
  'min': 0,
  'max': 1,
  'palette': ['red', 'green']
}

In [15]:
Map.addLayer(aoiCropland, bin_viz, 'Cropland in GFSAD', 1)
Map

Map(bottom=7678.0, center=[17.611001107213966, 8.08094566045597], controls=(WidgetControl(options=['position',…

In [17]:
arableFromCropland = aoiCropland.select('Cropland').unmask(0).clip(adm0_AOI)


-----------------
## **Hansen Dataset for cleared forested between 2000 and 2015**
-----------------

In [21]:
Hansen = Hansen_GFC.clipToCollection(adm0_AOI)
#Create Mask, selecting "loss year layer" less than or equal to 2015
aoiClearForestMask = Hansen.select("lossyear").lte(deforestCompYear)
#Mask Collection for only forests cleared between 2000 and 2015
aoiClearedForest = Hansen.mask(aoiClearForestMask)
aoiClearedForestLoss = aoiClearedForest.select('lossyear').gt(0).selfMask().rename("loss")
aoiClearedForestLoss = aoiClearedForestLoss.select('loss').unmask(0).clip(adm0_AOI)

In [22]:
Map.addLayer(aoiClearedForestLoss, bin_viz, 'forest loss', 1)
Map

Map(bottom=3955.0, center=[19.02057711096681, 5.603027343750001], controls=(WidgetControl(options=['position',…



-----------------
## **Global Man-made Impervious Surface (GMIS) Dataset**
-----------------

In [23]:
GMIS_ImperviousSurface = ImperviousSurfaceGMIS
impermeableSurface = GMIS_ImperviousSurface.lte(99).unmask(0).clip(adm0_AOI)
#Initialize a Map
Map = geemap.Map()
Map.centerObject(adm0_AOI)
Map.addLayer(impermeableSurface, bin_viz, 'impermeable Surface', 1)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…


-----------------
## **Global Surface Water Dataset**
-----------------

In [30]:
#Clip JRC Image to Aoi and select "occurrence" band and unmask image so that nonwater pixels are 0
surfaceWater_BFA = ee.Image("JRC/GSW1_4/GlobalSurfaceWater").select('occurrence').clip(adm0_AOI)
WaterOccuring = surfaceWater_BFA.gt(occurrence_threshold).unmask(0).clip(adm0_AOI)
water_viz = {
  'min': 0,
  'max': 1,
  'palette': ['white', 'blue']
}
#Initialize a Map
Map = geemap.Map()
Map.centerObject(adm0_AOI)
Map.addLayer(WaterOccuring, water_viz, 'Water Occuring', 1)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…



-----------------
## **World’s protected areas**
-----------------

In [33]:
#Initialize a Map
Map = geemap.Map()
Map.centerObject(adm0_AOI)
Map.addLayer(ProtectedAreaPolygons, {'color': 'red'}, 'Protected Areas polygons',1)
Map


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [34]:
Map.addLayer(ProtectedAreaPoints, {'color': 'red'}, 'Protected Areas points')
Map

Map(bottom=30857.0, center=[12.055437094469546, 2.3812866210937504], controls=(WidgetControl(options=['positio…

In [38]:
inProtectedArea = 1
outProtectedArea = 0
outProtectedAreaImage = ee.Image(outProtectedArea).clip(adm0_AOI)
intProtectedAreaImage = ee.Image(inProtectedArea).clip(ProtectedAreaPolygons)
ProtectedAreaImage = outProtectedAreaImage.where(**{'test':intProtectedAreaImage, 'value':intProtectedAreaImage})
ProtectedAreaImage = ProtectedAreaImage.clip(adm0_AOI)

In [39]:
Map.addLayer(ProtectedAreaImage, bin_viz, 'Protected Area Image', 1)
Map

Map(bottom=4029.0, center=[15.919073517982426, 4.899902343750001], controls=(WidgetControl(options=['position'…

In [41]:
intProtectedAreaImage = ee.Image(inProtectedArea).clip(ProtectedAreaPoints)
PointsToImage = outProtectedAreaImage.where(**{'test':intProtectedAreaImage, 'value':intProtectedAreaImage})
PointsToImage = PointsToImage.clip(adm0_AOI)

In [42]:
Map.addLayer(PointsToImage, bin_viz, 'Points  Image', 1)
Map

Map(bottom=4029.0, center=[15.919073517982426, 4.899902343750001], controls=(WidgetControl(options=['position'…

In [ ]:
aoiCropland = arableFromCropland.add(aoiClearedForestLoss).gte(1).selfMask().rename('arableLand')
arableLand_draft = aoiCropland.select('arableLand').unmask(0).clip(adm0_AOI)
arableLand_draft = arableLand_draft.subtract(ProtectedAreaImage).subtract(PointsToImage).subtract(WaterOccuring).subtract(impermeableSurface).rename('arableLand')



In [43]:
snippet = """
var aoiCropland = arableFromCropland.add(aoiClearedForestLoss).gte(1).selfMask().rename('arableLand');
var arableLand_draft = aoiCropland.select('arableLand').unmask(0).clip(adm0_AOI);
var arableLand_draft = arableLand_draft.subtract(ProtectedAreaImage).subtract(PointsToImage).subtract(WaterOccuring).subtract(impermeableSurface).rename('arableLand');

"""

geemap.js_snippet_to_py(snippet, add_new_cell=True, import_ee=False)


In [ ]:
aoiCropland = arableFromCropland.add(aoiClearedForestLoss).gte(1).selfMask().rename('arableLand')
arableLand_draft = aoiCropland.select('arableLand').unmask(0).clip(adm0_AOI)
arableLand_draft = arableLand_draft.subtract(ProtectedAreaImage).subtract(PointsToImage).subtract(WaterOccuring).subtract(impermeableSurface).rename('arableLand')

m

In [ ]:
intProtectedAreaImage = ee.Image(inProtectedArea).clip(ProtectedAreaPoints)
PointsToImage = outProtectedAreaImage.where(**{'test':intProtectedAreaImage, 'value':intProtectedAreaImage})
m